<a href="https://colab.research.google.com/github/yuho-ta/signate/blob/main/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import torch
from torchvision import datasets, models, transforms


In [ ]:
import pandas as pd
from PIL import Image

In [ ]:
# transformsの定義
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

In [ ]:
# dataset作成
image_datasets = {
    'train': datasets.ImageFolder('/content/drive/MyDrive/signate/練習　鋳造製品の欠陥検出/train_data/train',data_transforms['train']),
    'val': datasets.ImageFolder('/content/drive/MyDrive/signate/練習　鋳造製品の欠陥検出/train_data/val',data_transforms['val'])
}

# dataloaders作成
image_dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=4,shuffle=True, num_workers=0, drop_last=True),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=4,shuffle=False, num_workers=0, drop_last=True),
}


In [ ]:
import torch.nn as nn
import torch.optim as optim
device = "cpu"
TARGET_NUM = 2

# モデル作成関数の定義
def get_model(target_num,isPretrained=False):
    """for get model"""

    model_ft = models.resnet18(pretrained=isPretrained)
    model_ft.fc = nn.Linear(512, target_num)
    model_ft = model_ft.to(device)
    return model_ft

# モデルのインスタンス作成
model = get_model(TARGET_NUM,isPretrained=False)
# 最適化関数定義
optimizer = optim.SGD(model.parameters(),lr=0.001, momentum=0.9)

# loss関数定義
criterion = nn.CrossEntropyLoss()

In [ ]:
from torch.utils.data import Dataset
from PIL import Image, ImageFilter
# モデル学習用関数
def train_model(model, criterion, optimizer, num_epochs=2,is_saved = False):
    best_acc = 0.0

    # エポック数だけ下記工程の繰り返し
    for epoch in range(num_epochs):

        for phase in ['train', 'val']:
            print('{}:フェイズ'.format(phase))

            # 訓練フェイズと検証フェイズの切り替え
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # dataloadersからバッチサイズだけデータ取り出し、下記工程（1−5）の繰り返し
            for i,(inputs, labels) in enumerate(image_dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 1. optimizerの勾配初期化
                optimizer.zero_grad()

                # 2.モデルに入力データをinputし、outputを取り出す
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                print(preds)
                # 3. outputと正解ラベルから、lossを算出
                loss = criterion(outputs, labels)
                print('   loaders:{}回目'.format(i+1)  ,'   loss:{}'.format(loss))

                if phase == 'train':
                    # 4. 誤差逆伝播法により勾配の算出
                    loss.backward()
                    # 5. optimizerのパラメータ更新
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            dataset_sizes = len(image_datasets[phase])
            epoch_loss = running_loss / dataset_sizes
            epoch_acc = running_corrects.double() / dataset_sizes
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # C. 今までのエポックでの精度よりも高い場合はモデルの保存
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                if(is_saved):
                    torch.save(model.state_dict(), '/content/drive/MyDrive/signate/練習　鋳造製品の欠陥検出/original_model_{}.pth'.format(epoch))

    print('Best val Acc: {:4f}'.format(best_acc))
train_model(model, criterion, optimizer, num_epochs=10,is_saved =True)

train:フェイズ
tensor([0, 0, 0, 0])
   loaders:1回目    loss:0.8297223448753357
tensor([0, 0, 0, 0])
   loaders:2回目    loss:0.688908576965332
tensor([0, 0, 0, 0])
   loaders:3回目    loss:0.6686077117919922
tensor([0, 0, 0, 0])
   loaders:4回目    loss:0.6659454107284546
tensor([0, 1, 1, 1])
   loaders:5回目    loss:0.7042588591575623
tensor([1, 1, 1, 1])
   loaders:6回目    loss:0.7496195435523987
tensor([0, 0, 0, 0])
   loaders:7回目    loss:0.6180129647254944
tensor([0, 0, 0, 0])
   loaders:8回目    loss:0.6462961435317993
tensor([0, 0, 0, 0])
   loaders:9回目    loss:1.1236010789871216
tensor([0, 0, 0, 0])
   loaders:10回目    loss:0.8421973586082458
tensor([0, 0, 0, 0])
   loaders:11回目    loss:0.8098795413970947
tensor([0, 0, 0, 0])
   loaders:12回目    loss:0.6600954532623291
tensor([0, 0, 0, 0])
   loaders:13回目    loss:0.616813600063324
tensor([0, 0, 0, 0])
   loaders:14回目    loss:0.7001818418502808
tensor([1, 0, 0, 0])
   loaders:15回目    loss:0.7229180335998535
tensor([1, 1, 1, 1])
   loaders:16回目    

In [ ]:
import re
test = pd.DataFrame(data= os.listdir('/content/drive/MyDrive/signate/練習　鋳造製品の欠陥検出/test_data'))
test = test.rename(columns={0: 'filename'})
test['filename'] = sorted(test['filename'], key=lambda x: int(re.search(r'(\d+)', x).group()))
# targetカラム作成
test['target'] = 0

In [ ]:
def get_model(target_num,isPretrained=False):
    model_ft = models.resnet18(pretrained=isPretrained)
    model_ft.fc = nn.Linear(512, target_num)
    model_ft = model_ft.to(device)
    return model_ft
# 事前学習済みResNet18モデルのインスタンス作成
pretrained_model = get_model(target_num=TARGET_NUM)
pretrained_model.load_state_dict(torch.load('/content/drive/MyDrive/signate/練習　鋳造製品の欠陥検出/original_model_9.pth', map_location=lambda storage, loc: storage), strict=True)

<All keys matched successfully>

In [ ]:
# 推論用のtransforms作成
test_transforms = transforms.Compose([
               transforms.Resize(256),
               transforms.ToTensor(),
               transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),

])

# dataset作成
class Test_Datasets(Dataset):

    def __init__(self, data_transform):

        self.df = test
        self.data_transform = data_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        file = self.df['filename'][index]
        image = Image.open('/content/drive/MyDrive/signate/練習　鋳造製品の欠陥検出/test_data/'+ file)
        image = self.data_transform(image)
        return image,file



In [ ]:
# datasetのインスタンス作成
test_dataset = Test_Datasets(data_transform=test_transforms)

# dataloader作成
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=1,
                                                shuffle=False,
                                                num_workers=0 ,
                                                drop_last=True )

In [ ]:
pred = []  # 予測を保存するリストを初期化
for i,(inputs, labels) in enumerate(test_dataloader):

    inputs = inputs.to(device)

    # 学習済みモデルを推論モードに設定
    pretrained_model.eval()

    # 学習済みモデルにデータをインプットし、推論をさせる
    outputs = pretrained_model(inputs)
    print(outputs)
    # アウトプットから推定されたラベルを取得
    _, preds = torch.max(outputs, 1)

    # 事前に用意したリストに推論結果(0 or 1)を格納
    pred.append(preds.item())
test['target'] = pred
print(pred)
test.to_csv('/content/drive/MyDrive/signate/練習　鋳造製品の欠陥検出/test.csv',  header=False, index=False)

tensor([[-0.0500,  1.1725]], grad_fn=<AddmmBackward0>)
tensor([[-0.5890,  1.9858]], grad_fn=<AddmmBackward0>)
tensor([[-1.7078,  3.1974]], grad_fn=<AddmmBackward0>)
tensor([[ 3.2199, -1.9347]], grad_fn=<AddmmBackward0>)
tensor([[-2.0474,  3.7534]], grad_fn=<AddmmBackward0>)
tensor([[ 1.7781, -0.6087]], grad_fn=<AddmmBackward0>)
tensor([[ 2.2909, -1.0922]], grad_fn=<AddmmBackward0>)
tensor([[ 2.0962, -0.9204]], grad_fn=<AddmmBackward0>)
tensor([[-0.3222,  1.6100]], grad_fn=<AddmmBackward0>)
tensor([[ 2.8673, -1.6062]], grad_fn=<AddmmBackward0>)
tensor([[ 1.7965, -0.5781]], grad_fn=<AddmmBackward0>)
tensor([[ 2.0405, -1.0117]], grad_fn=<AddmmBackward0>)
tensor([[ 2.8128, -1.5038]], grad_fn=<AddmmBackward0>)
tensor([[ 2.3766, -1.3605]], grad_fn=<AddmmBackward0>)
tensor([[-0.6013,  2.0333]], grad_fn=<AddmmBackward0>)
tensor([[ 2.0360, -0.8492]], grad_fn=<AddmmBackward0>)
tensor([[ 2.9670, -1.5960]], grad_fn=<AddmmBackward0>)
tensor([[-0.0020,  1.5293]], grad_fn=<AddmmBackward0>)
tensor([[ 